In [1]:
from math import log, exp
import pandas as pd
import nltk
from n_gram_updated import *

Importing and Processing the Test and Train Data

In [2]:
filepath = "train_data.csv"
df_train = pd.read_csv(filepath)
df_train.rename(columns = {df_train.columns[0]:"Value"},inplace=True)
df_train.size

263000

In [3]:
filepath = "test_data.csv"
df_test = pd.read_csv(filepath)
df_test.rename(columns = {df_test.columns[0]:"Value"},inplace=True)
df_test.size

65750

In [4]:
# Train Corpus
train_corpus = {}
sentence_id = 0
for sentence in df_train['Value']:
    tokenized_sentence = []
    tokens = nltk.word_tokenize(sentence)
    tokens = (token for token in tokens if token.isalpha())
    tokenized_sentence += tokens
    train_corpus[sentence_id] = tokenized_sentence
    sentence_id += 1

In [5]:
# Test Corpus
test_corpus = {} 
sentence_id = 0
for sentence in df_test['Value']:
    tokenized_sentence = []
    tokens = nltk.word_tokenize(sentence)
    tokens = (token for token in tokens if token.isalpha())
    tokenized_sentence += tokens
    test_corpus[sentence_id] = tokenized_sentence
    sentence_id += 1

In [6]:
# Defining Vocabulary

Vocab = vocabulary(train_corpus)
V = len(Vocab)
print(V)

66112


In [7]:
# count_dict = count_n_gram(train_corpus,2)
# Nc_ = Nc(count_dict)
# print(Nc_[0])

In [8]:
# import numpy as np

# def interpolate_with_numpy(lst):
#     # Convert list to numpy array for convenience
#     arr = np.array(lst)
    
#     # Find indices of non-zero values
#     non_zero_indices = np.nonzero(arr)[0]
    
#     # Handle case where the list starts with zeros
#     if non_zero_indices[0] != 0:
#         non_zero_indices = np.insert(non_zero_indices, 0, 0)
    
#     # Interpolate zeros (excluding the last value)
#     arr[:-1] = np.interp(np.arange(len(arr)-1), non_zero_indices, arr[non_zero_indices])
    
#     # Extrapolate the last value
#     last_non_zero = arr[-2]  # As per your condition the second last value is always non-zero
#     second_last_non_zero = arr[non_zero_indices[-2]]
#     gradient = last_non_zero - second_last_non_zero
#     arr[-1] = last_non_zero + gradient
#     # arr = list(arr)
#     return arr.tolist()

# # Example
# lst = [0, 0, 2, 0, 4, 6, 0]
# print(interpolate_with_numpy(lst))

# lst = Nc_
# lst1 = interpolate_with_numpy(lst)
# lst2 = remove_null(Nc_)
# print(Nc_)
# # print(interpolate_with_numpy(lst))
# print(lst1)
# print(lst2)
# print(lst1==lst2)

Unigram Model

In [9]:
prob_unigram = train_n_gram(train_corpus, 1)
pp, pp_unigram = test_n_gram(test_data=test_corpus,n=1, prob_words=prob_unigram)
print("Perplexity for Unigram model without smoothing:", pp_unigram)

Perplexity for Unigram model without smoothing: 255029160.07513666


Unigram Model with Laplace Smoothing

In [10]:
prob_unigram_sm = train_n_gram(train_corpus, 1, vocab = V, smoothing=True,how='Laplace')
pp, pp_unigram_sm = test_n_gram(test_data=test_corpus,n=1, prob_words=prob_unigram_sm, Vocabulary=V, smoothing=True, how='Laplace', processed_corpus=train_corpus)
print("Perplexity for Unigram model with Laplace smoothing:", pp_unigram_sm)

Perplexity for Unigram model with Laplace smoothing: 1270.0001626416067


Unigram Model with Add-k Smoothing

In [11]:
K_uni = 0
for i in range(30):
    if K_uni < 1:
        K_uni += 0.1
        K_uni = round(K_uni, 1)
    elif K_uni < 10:
        K_uni += 1
        K_uni = int(K_uni)
    else:
        K_uni += 10
    prob_unigram_sm_addk = train_n_gram(train_corpus, 1, vocab = V, smoothing=True,how='Add_k',k=K_uni)
    pp, pp_unigram_sm_addk = test_n_gram(test_data=test_corpus,n=1, prob_words=prob_unigram_sm_addk, Vocabulary=V, smoothing=True, how='Add_k', k=K_uni, processed_corpus=train_corpus)
    print("Perplexity for Unigram model with Add-k smoothing:", pp_unigram_sm_addk, K_uni)

Perplexity for Unigram model with Add-k smoothing: 1688.9715811695312 0.1
Perplexity for Unigram model with Add-k smoothing: 1519.5908690793046 0.2
Perplexity for Unigram model with Add-k smoothing: 1442.7910037763445 0.3
Perplexity for Unigram model with Add-k smoothing: 1395.2464936658516 0.4
Perplexity for Unigram model with Add-k smoothing: 1361.5085128295998 0.5
Perplexity for Unigram model with Add-k smoothing: 1335.6600505660454 0.6
Perplexity for Unigram model with Add-k smoothing: 1314.8652195956113 0.7
Perplexity for Unigram model with Add-k smoothing: 1297.5637837565068 0.8
Perplexity for Unigram model with Add-k smoothing: 1282.812422347832 0.9
Perplexity for Unigram model with Add-k smoothing: 1270.0001626416067 1.0
Perplexity for Unigram model with Add-k smoothing: 1194.1445563314903 2
Perplexity for Unigram model with Add-k smoothing: 1156.8848198783232 3
Perplexity for Unigram model with Add-k smoothing: 1134.3689566358137 4
Perplexity for Unigram model with Add-k smoot

Unigram Model with Good Turing

In [12]:
prob_unigram_GT = train_n_gram(train_corpus, n=1, how='Good Turing')
pp, pp_unigram_GT = test_n_gram(test_data=test_corpus, n=1, prob_words=prob_unigram_GT, how='Good Turing')
print("Perplexity for Unigram model with Good Turing smoothing:", pp_unigram_GT)

Perplexity for Unigram model with Good Turing smoothing: 968.5055318410206


Bigram Model

In [13]:
prob_bigram = train_n_gram(train_corpus, 2)
pp, pp_bigram = test_n_gram(test_data=test_corpus,n=2, prob_words=prob_bigram)
print("Perplexity for Bigram model without smoothing:", pp_bigram)

Perplexity for Bigram model without smoothing: 31069869118.23457


Bigram Model with Laplace Smoothing

In [14]:
prob_bigram_sm = train_n_gram(train_corpus, 2, vocab = V, smoothing=True, how='Laplace')
pp, pp_bigram_sm = test_n_gram(test_data=test_corpus,n=2, prob_words=prob_bigram_sm, Vocabulary=V, smoothing=True, how='Laplace', processed_corpus=train_corpus)
print("Perplexity for Bigram model with Laplace smoothing:", pp_bigram_sm)

Perplexity for Bigram model with Laplace smoothing: 1813.9033369573867


Bigram Model with Add-k Smoothing

In [15]:
prob_bigram_sm_addk = train_n_gram(train_corpus, 2, vocab = V, smoothing=True,how='Add_k',k=0.01)
pp, pp_bigram_sm_addk = test_n_gram(test_data=test_corpus,n=2, prob_words=prob_bigram_sm_addk, Vocabulary=V, smoothing=True, how='Add_k', k=0.01, processed_corpus=train_corpus)
print("Perplexity for Bigram model with Add-k smoothing:", pp_bigram_sm_addk)

Perplexity for Bigram model with Add-k smoothing: 524.2167864319489


Bigram Model with Good Turing

In [16]:
prob_bigram_GT = train_n_gram(train_corpus, n=2, how='Good Turing')
pp, pp_bigram_GT = test_n_gram(test_data=test_corpus, n=2, prob_words=prob_bigram_GT, how='Good Turing')
print("Perplexity for Bigram model with Good Turing smoothing:", pp_bigram_GT)

Perplexity for Bigram model with Good Turing smoothing: 26041.65749218391


Trigram Model

In [17]:
prob_trigram = train_n_gram(train_corpus, 3)
pp, pp_trigram = test_n_gram(test_data=test_corpus,n=3, prob_words=prob_trigram)
print("Perplexity for Trigram model without smoothing:", pp_trigram)

Perplexity for Trigram model without smoothing: 69139995101.51898


Trigram Model with Laplace Smoothing

In [18]:
prob_trigram_sm = train_n_gram(train_corpus, 3, vocab = V, smoothing=True, how='Laplace')
pp, pp_trigram_sm = test_n_gram(test_data=test_corpus,n=3, prob_words=prob_trigram_sm, Vocabulary=V, smoothing=True, how='Laplace', processed_corpus=train_corpus)
print("Perplexity for Trigram model with Laplace smoothing:", pp_trigram_sm)

Perplexity for Trigram model with Laplace smoothing: 4466.5111161327695


Trigram Model with Add-k Smoothing

In [19]:
prob_trigram_sm_addk = train_n_gram(train_corpus, 3, vocab = V, smoothing=True,how='Add_k',k=0.01)
pp, pp_trigram_sm_addk = test_n_gram(test_data=test_corpus,n=3, prob_words=prob_trigram_sm_addk, Vocabulary=V, smoothing=True, how='Add_k', k=0.01, processed_corpus=train_corpus)
print("Perplexity for Trigram model with Add-k smoothing:", pp_trigram_sm_addk)

Perplexity for Trigram model with Add-k smoothing: 1301.4960365315192


Trigram Model with Good Turing

In [20]:
prob_trigram_GT = train_n_gram(train_corpus, n=3, how='Good Turing')
pp, pp_trigram_GT = test_n_gram(test_data=test_corpus, n=3, prob_words=prob_trigram_GT, how='Good Turing')
print("Perplexity for Trigram model with Good Turing smoothing:", pp_trigram_GT)

Perplexity for Trigram model with Good Turing smoothing: 63173.0338193803


Quadgram Model

In [21]:
prob_quadgram = train_n_gram(train_corpus, 4)
pp, pp_quadgram = test_n_gram(test_data=test_corpus,n=4, prob_words=prob_quadgram)
print("Perplexity for Quadgram model without smoothing:", pp_quadgram)

Perplexity for Quadgram model without smoothing: 292224555035.2305


Quadgram Model with Laplace Smoothing

In [22]:
prob_quadgram_sm = train_n_gram(train_corpus, 4, vocab = V, smoothing=True, how='Laplace')
pp, pp_quadgram_sm = test_n_gram(test_data=test_corpus,n=4, prob_words=prob_quadgram_sm, Vocabulary=V, smoothing=True, how='Laplace', processed_corpus=train_corpus)
print("Perplexity for Quadgram model with Laplace smoothing:", pp_quadgram_sm)

Perplexity for Quadgram model with Laplace smoothing: 5013.931269898044


Quadgram Model with Add-k Smoothing

In [23]:
prob_quadgram_sm_addk = train_n_gram(train_corpus, 4, vocab = V, smoothing=True,how='Add_k',k=0.01)
pp, pp_quadgram_sm_addk = test_n_gram(test_data=test_corpus,n=4, prob_words=prob_quadgram_sm_addk, Vocabulary=V, smoothing=True, how='Add_k', k=0.01, processed_corpus=train_corpus)
print("Perplexity for Quadgram model with Add-k smoothing:", pp_quadgram_sm_addk)

Perplexity for Quadgram model with Add-k smoothing: 2362.410302953623


Quadgram Model with Good Turing

In [24]:
prob_quadgram_GT = train_n_gram(train_corpus, n=4, how='Good Turing')
pp, pp_quadgram_GT = test_n_gram(test_data=test_corpus, n=4, prob_words=prob_quadgram_GT, how='Good Turing')
print("Perplexity for Quadgram model with Good Turing smoothing:", pp_quadgram_GT)

Perplexity for Quadgram model with Good Turing smoothing: 87033.95812694842


Perplexities of different Models on training data

In [25]:

pp, pp_unigram_train = test_n_gram(test_data=train_corpus,n=1, prob_words=prob_unigram)
print("Perplexity for Unigram model on training data:", pp_unigram_train)

pp, pp_bigram_train = test_n_gram(test_data=train_corpus,n=2, prob_words=prob_bigram)
print("Perplexity for Bigram model on training data:", pp_bigram_train)

pp, pp_trigram_train = test_n_gram(test_data=train_corpus,n=3, prob_words=prob_trigram)
print("Perplexity for Trigram model on training data:", pp_trigram_train)

pp, pp_quadgram_train = test_n_gram(test_data=train_corpus,n=4, prob_words=prob_quadgram)
print("Perplexity for Quadgram model on training data:", pp_quadgram_train)

Perplexity for Unigram model on training data: 1236.976795496186
Perplexity for Bigram model on training data: 76.80480458054907
Perplexity for Trigram model on training data: 8.646154355512213
Perplexity for Quadgram model on training data: 2.9718372121761303
